# 기상데이터 병합 및 일자별 평균

* 각 관측소마다 일시별로 측정된 데이터를 평균내고, 이를 다시 지역별로 평균

In [74]:
# 필요 라이브러리 import
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
from urllib import parse
import os

In [89]:
data = pd.read_excel('data/미세먼지/2019년 7월.xlsx')
data.head(3)

,지역,망,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소
0,서울 중구,도시대기,111121,중구,2019070101,0.002,0.5,0.055,0.010,40.0,29.0,서울 중구 덕수궁길 15
1,서울 중구,도시대기,111121,중구,2019070102,0.002,0.5,0.056,0.012,36.0,29.0,서울 중구 덕수궁길 15
2,서울 중구,도시대기,111121,중구,2019070103,0.003,0.7,0.055,0.011,37.0,30.0,서울 중구 덕수궁길 15


In [90]:
data = data.loc[(data['망'] == '도시대기')].reset_index(drop=True)

In [91]:
data['측정날짜'] = data['측정일시'].astype('str').str[:-2]

data['측정일시'] = data['측정일시'].astype('str').str[-2:].replace('24', '00')

In [92]:
data['지역'] = data['지역'].str[:2]

In [93]:
data.groupby(['측정날짜', '지역']).mean().iloc[:, 1:].reset_index()

,측정날짜,지역,SO2,CO,O3,NO2,PM10,PM25
0,20190701,강원,0.002070,0.405010,0.051774,0.007813,40.009921,26.654088
1,20190701,경기,0.003367,0.504278,0.054839,0.013144,38.052950,24.923656
2,20190701,경남,0.003708,0.534396,0.034113,0.015659,47.148148,27.337268
3,20190701,경북,0.002876,0.394953,0.037396,0.010912,39.955654,23.721228
4,20190701,광주,0.002179,0.433333,0.028042,0.015333,34.540881,21.222892
...,...,...,...,...,...,...,...,...
522,20190731,전남,0.002905,0.313221,0.015729,0.004418,13.352792,5.286733
523,20190731,전북,0.002476,0.209478,0.016617,0.005101,12.981781,5.322404
524,20190731,제주,0.002188,0.191860,0.017467,0.005092,13.781250,7.830357
525,20190731,충남,0.002834,0.308100,0.020247,0.004093,14.683183,5.497660


In [107]:
## 함수화


pollution = pd.DataFrame()
base_path = 'data/미세먼지/'
file_list = os.listdir('data/미세먼지/')

for file in tqdm(file_list):

    # 파일 경로 결합 및 불러오기
    file_path = base_path + file
    data = pd.read_excel(file_path)
    
    # 도시 대기만 사용
    data = data.loc[(data['망'] == '도시대기')].reset_index(drop=True)
    
    data['측정날짜'] = data['측정일시'].astype('str').str[:-2]
    data.drop('측정일시', axis=1, inplace=True)
    data['행정구역'] = data['지역'].str[:2]
    
    data.groupby(['측정날짜', '행정구역']).mean()
    
    grouped_data = data.groupby(['측정날짜', '행정구역']).mean().iloc[:, 1:].reset_index()
    
    pollution = pd.concat([pollution, grouped_data], axis=0)
    
    

In [110]:
pollution = np.round(pollution, 3)

In [113]:
pollution.sort_values('측정날짜').to_csv('data/병합데이터/지역별미세먼지데이터.csv', encoding='utf-8')